In [1]:
from __future__ import print_function
#from sklearn.cross_validation import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

Using TensorFlow backend.


In [2]:
traindata = pd.read_csv('Training.csv', header=None)
testdata = pd.read_csv('Testing.csv', header=None)

In [3]:
X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,0]
C = testdata.iloc[:,0]
T = testdata.iloc[:,1:42]

In [4]:
scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
print(trainX[0:5,:])

[[0.000e+00 6.515e-04 6.515e-04 6.515e-04 1.968e-01 9.519e-01 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 6.515e-04 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 3.909e-03 3.909e-03 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  6.515e-04 0.000e+00 0.000e+00 1.661e-01 1.661e-01 6.515e-04 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 1.224e-04 1.224e-04 1.224e-04 1.861e-02 9.989e-01 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.224e-04 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 2.448e-04 2.448e-04 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  1.224e-04 0.000e+00 0.000e+00 3.122e-02 3.122e-02 1.224e-04 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 6.801e-04 6.801e-04 6.801e-04 1.510e-01 9.576e-01 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 6.801e-04 0.000e+00 0.000e+00
  0.000e+0

In [5]:
scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
print(testT[0:5,:])

[[0.000e+00 1.834e-04 1.834e-04 1.834e-04 3.319e-02 9.994e-01 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.834e-04 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 1.467e-03 1.467e-03 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  1.834e-04 0.000e+00 0.000e+00 1.650e-03 1.650e-03 1.834e-04 0.000e+00
  2.017e-05 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 1.844e-03 1.844e-03 1.844e-03 4.407e-01 8.961e-01 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.844e-03 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 1.475e-02 1.475e-02 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  1.844e-03 0.000e+00 0.000e+00 3.503e-02 3.503e-02 1.844e-03 0.000e+00
  9.219e-05 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 7.363e-04 7.363e-04 7.363e-04 1.730e-01 9.844e-01 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 7.363e-04 0.000e+00 0.000e+00
  0.000e+0

In [6]:
y_train = np.array(Y)
y_test = np.array(C)

In [7]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))


In [8]:
print(X_train.shape)

(493715, 1, 41)


In [9]:
batch_size = 32

In [10]:
# 1. define the network
model = Sequential()
model.add(LSTM(4,input_dim=41))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))
print(model.get_config())


/Users/muhammedsara/anaconda3/envs/deeplearning3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/muhammedsara/anaconda3/envs/deeplearning3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(4, input_shape=(None, 41))`
  This is separate from the ipykernel package so we can avoid doing imports until


{'name': 'sequential_1', 'layers': [{'class_name': 'LSTM', 'config': {'name': 'lstm_1', 'trainable': True, 'batch_input_shape': (None, None, 41), 'dtype': 'float32', 'return_sequences': False, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'units': 4, 'activation': 'tanh', 'recurrent_activation': 'hard_sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1.0, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'unit_forget_bias': True, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, 'dropout': 0.0, 'recurrent_dropout': 0.0, 'implementation': 1}}, {'class_name': 'Dropout', 'config': {'name': 'dro

In [11]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="results/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('results/training_set_iranalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=10, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
model.save("results/lstm1layer_model.hdf5")

loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
y_pred = model.predict_classes(X_test)
np.savetxt('lstm1predicted.txt', y_pred, fmt='%01d')



/Users/muhammedsara/anaconda3/envs/deeplearning3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Train on 493715 samples, validate on 306 samples
Epoch 1/10
493715/493715 [==============================] - 157s 318us/step - loss: 0.0608 - acc: 0.9785 - val_loss: 0.0144 - val_acc: 1.0000

Epoch 00001: val_acc improved from -inf to 1.00000, saving model to checkpoint-01.hdf5
Epoch 2/10
493715/493715 [==============================] - 126s 256us/step - loss: 0.0290 - acc: 0.9882 - val_loss: 0.0148 - val_acc: 0.9967

Epoch 00002: val_acc did not improve from 1.00000
Epoch 3/10
493715/493715 [==============================] - 134s 271us/step - loss: 0.0258 - acc: 0.9897 - val_loss: 0.0167 - val_acc: 1.0000

Epoch 00003: val_acc did not improve from 1.00000
Epoch 4/10
493715/493715 [==============================] - 108s 219us/step - loss: 0.0236 - acc: 0.9905 - val_loss: 0.0180 - val_acc: 1.0000

Epoch 00004: val_acc did not improve from 1.00000
Epoch 5/10
493715/493715 [==============================] - 94s 190us/step - loss: 0.0214 - acc: 0.9920 - val_loss: 0.0164 - val_acc: 0.9967

